ON CHARGE LES CORRESPONDANCES INSEE / CODE POSTAL

In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import seaborn as sns
import datetime
import requests
import random
from dotenv import load_dotenv   #for python-dotenv method
load_dotenv()                    #for python-dotenv method
import os


data_insee = pd.read_csv("src/insee.csv", sep=";", dtype='str')

In [120]:
data_insee.head()

,#Code_commune_INSEE,Nom_de_la_commune,Code_postal,Libellé_d_acheminement,Ligne_5
0,01001,L ABERGEMENT CLEMENCIAT,01400,L ABERGEMENT CLEMENCIAT,NaN
1,01002,L ABERGEMENT DE VAREY,01640,L ABERGEMENT DE VAREY,NaN
2,01004,AMBERIEU EN BUGEY,01500,AMBERIEU EN BUGEY,NaN
3,01005,AMBERIEUX EN DOMBES,01330,AMBERIEUX EN DOMBES,NaN
4,01006,AMBLEON,01300,AMBLEON,NaN


ON VEUT QUE LES VILLES QUI SONT EN ILE DE FRANCE

In [121]:

idf=['93','75','77','92', '78','91','94','95']
df_idf = pd.DataFrame(columns=('insee','code_postal','nom'))
for i in range(len(data_insee)-1):
    code = str(data_insee.iloc[i,2])[0:2]
    if  code in idf:
        insee = data_insee.iloc[i,0]
        code_postal = data_insee.iloc[i,2]
        nom= data_insee.iloc[i,1]
        df_idf.loc[len(df_idf)+1]= (insee, code_postal, nom)
    

In [122]:
filename = 'backup_idf_avec_coordonnees.csv'

if filename in os.listdir("csv/"):
    on_a_deja_les_coordonnees = True
    print("le fichier existe")
    df_idf_avec_coordonnees = pd.DataFrame(pd.read_csv('csv/'+filename), dtype='str')
else:
    print("Le fichier existe pas")
    on_a_deja_les_coordonnees = False


Le fichier existe pas


ON RECUPERE LES COORDONNEES (LATITUDE ET LONGITUDE) DE TOUTES LES VILLES D'IDF

In [123]:

if(on_a_deja_les_coordonnees==False):
    # RECUPERATION DES DONNEES POUR TOUTE LA LISTE DES VILLES
    url = "https://nominatim.openstreetmap.org/search?"
    headers = {'user_agent':'charles.mittempergher@gmail.com'}

    df_idf_avec_coordonnees=pd.DataFrame(columns=("insee", "code_postal", "nom", "lat", "lon"))

    for i in range(0,len(df_idf)):
        
        p = {'postalcode' : {df_idf.iloc[i,1]},'country':'France', 'format':'json', 'limit':1, 'email':'charles@gmail.com'}
        r = requests.get(url, headers=headers, params=p)
        lat   = r.json()[0]['lat']
        lon   = r.json()[0]['lon']
        code_postal= df_idf.iloc[i,1]
        insee = df_idf.iloc[i,0]
        nom = df_idf.iloc[i,2]

        df_idf_avec_coordonnees.loc[i] =(insee, code_postal, nom, lat, lon)

        print(f'{i}/{len(df_idf)} ({round(i/len(df_idf)*100,2)}%)')
        print('*************************')

0/1382 (0.0%)
*************************
1/1382 (0.0723589001447178%)
*************************
2/1382 (0.1447178002894356%)
*************************
3/1382 (0.21707670043415342%)
*************************
4/1382 (0.2894356005788712%)
*************************
5/1382 (0.361794500723589%)
*************************
6/1382 (0.43415340086830684%)
*************************
7/1382 (0.5065123010130246%)
*************************
8/1382 (0.5788712011577424%)
*************************
9/1382 (0.6512301013024602%)
*************************
10/1382 (0.723589001447178%)
*************************
11/1382 (0.7959479015918958%)
*************************
12/1382 (0.8683068017366137%)
*************************
13/1382 (0.9406657018813315%)
*************************
14/1382 (1.0130246020260492%)
*************************
15/1382 (1.085383502170767%)
*************************
16/1382 (1.1577424023154848%)
*************************
17/1382 (1.2301013024602026%)
*************************
18/1382 (1.3024602

ON CONVERTIT LES COLONNES LATITUDE ET LONGITUDE EN FLOAT POUR POUVOIR LES UTILISER SUR LA CARTE 

In [124]:
df_idf_avec_coordonnees['lat'] = df_idf_avec_coordonnees['lat'].astype('float64')
df_idf_avec_coordonnees['lon'] = df_idf_avec_coordonnees['lon'].astype('float64')

ON EST CONTENT ALORS ON FAIT UNE SAUVEGARDE INTERMEDIAIRE EN CSV

In [125]:
if(on_a_deja_les_coordonnees==False):
    df_idf_avec_coordonnees.to_csv("csv/backup_idf_avec_coordonnees.csv", index=False)

TEST DE L'API AIRPARIF AVEC UNE SEULE VILLE

In [126]:
url = "https://api.airparif.fr/indices/prevision/commune"
headers = {'X-Api-Key': os.environ.get('airparif')}
p = {'insee' : '93010'}

response = requests.get(url, headers=headers, params=p)

In [127]:
df_idf_avec_coordonnees.iloc[0,0]

'75101'

RECUPERATION DES ESTIMATIONS POUR LES PROCHAINS JOURS POUR TOUTES LES VILLES D'IDF

In [128]:
response.json().get('75101')

In [129]:
df_idf_avec_coordonnees.iloc[0,0]

'75101'

In [130]:
df_idf_avec_coordonnees.head

<bound method NDFrame.head of       insee code_postal                  nom        lat       lon
0     75101       75001             PARIS 01  48.861878  2.337414
1     75102       75002             PARIS 02  48.867683  2.343129
2     75103       75003             PARIS 03  48.862686  2.358687
3     75104       75004             PARIS 04  48.854148  2.356808
4     75105       75005             PARIS 05  48.845419  2.352582
...     ...         ...                  ...        ...       ...
1377  95676       95510   VILLERS EN ARTHIES  49.087770  1.705854
1378  95678       95840        VILLIERS ADAM  49.066169  2.238902
1379  95680       95400      VILLIERS LE BEL  48.999480  2.402407
1380  95682       95720      VILLIERS LE SEC  49.050139  2.395147
1381  95690       95420  WY DIT JOLI VILLAGE  49.133450  1.782679

[1382 rows x 5 columns]>

SI ON A DEJA RECUPERE LES DONNEES DU JOUR ON RECHARGE LE CSV PLUTOT QUE FAIRE UN NOUVEL APPEL API

In [131]:
filename_data = f'{datetime.datetime.today().strftime('%Y-%m-%d')}-backup_data_valeurs.csv'

if filename_data in os.listdir("csv//"):
    print("le fichier existe")
    on_a_deja_les_datas_du_jour = True
    data_valeurs = pd.DataFrame(pd.read_csv('csv/'+filename_data), dtype='str')
else:
    on_a_deja_les_datas_du_jour = False
    print("Le fichier existe pas")

Le fichier existe pas


SI ON A PAS DE DONNEES DU JOUR ON UTILISE L'API ET ON SAUVEGARDE EN CSV

In [132]:

if (on_a_deja_les_datas_du_jour==False):

    url = "https://api.airparif.fr/indices/prevision/commune"
    headers = {'X-Api-Key': os.environ.get('airparif')}

    data_valeurs = pd.DataFrame(columns=('insee','code_postal','ville','lat','lon','date','no2','o3','pm10','pm25','so2','indice'))
    for i in range(0,len(df_idf_avec_coordonnees)):
        p = {'insee' : df_idf_avec_coordonnees.iloc[i,0]}
        response = requests.get(url, headers=headers, params=p)
        for r in response.json().get(df_idf_avec_coordonnees.iloc[i,0]):
            
            ville = df_idf_avec_coordonnees.iloc[i,2]
            code_postal=df_idf_avec_coordonnees.iloc[i,1]
            insee=df_idf_avec_coordonnees.iloc[i,0]
            lat=df_idf_avec_coordonnees.iloc[i,3]
            lon=df_idf_avec_coordonnees.iloc[i,4]
            date = r['date']
            no2 = r['no2']
            o3 = r['o3']
            pm10 = r['pm10']
            pm25 = r['pm25']
            so2 = r['so2']
            indice = r['indice']
            data_valeurs.loc[len(data_valeurs)+1]=(insee, code_postal,ville,lat,lon,date,no2,o3,pm10,pm25,so2,indice)
        print(df_idf_avec_coordonnees.iloc[i,1])
        print(f'{i}/{len(df_idf_avec_coordonnees)} ({i/len(df_idf_avec_coordonnees)*100}%)')
        print("*"*50)

75001
0/1382 (0.0%)
**************************************************
75002
1/1382 (0.0723589001447178%)
**************************************************
75003
2/1382 (0.1447178002894356%)
**************************************************
75004
3/1382 (0.21707670043415342%)
**************************************************
75005
4/1382 (0.2894356005788712%)
**************************************************
75006
5/1382 (0.361794500723589%)
**************************************************
75007
6/1382 (0.43415340086830684%)
**************************************************
75008
7/1382 (0.5065123010130246%)
**************************************************
75009
8/1382 (0.5788712011577424%)
**************************************************
75010
9/1382 (0.6512301013024602%)
**************************************************
75011
10/1382 (0.723589001447178%)
**************************************************
75012
11/1382 (0.7959479015918958%)
********************************

ON EST CONTENT ALORS ON FAIT UNE NOUVELLE SAUVEGARDE DU CSV

In [133]:
if (on_a_deja_les_datas_du_jour==False):
    data_valeurs.to_csv(f"csv/{datetime.datetime.today().strftime('%Y-%m-%d')}-backup_data_valeurs.csv", index=False)

POUR POUVOIR UTILISER LES RESULTATS ON LES TRANSFORME EN VALEURS NUMERIQUES ORDONNEES

In [134]:
mapping = {'Bon':4,'Moyen':3,'Mauvais':2,'Dégradé':1}

data_valeurs['no2'] = data_valeurs['no2'].map(mapping)
data_valeurs['o3'] = data_valeurs['o3'].map(mapping)
data_valeurs['pm10'] = data_valeurs['pm10'].map(mapping)
data_valeurs['pm25'] = data_valeurs['pm25'].map(mapping)
data_valeurs['so2'] = data_valeurs['so2'].map(mapping)
data_valeurs['indice'] = data_valeurs['indice'].map(mapping)


In [135]:
data_valeurs['lat'] = data_valeurs['lat'].astype('float64')
data_valeurs['lon'] = data_valeurs['lon'].astype('float64')

In [136]:
data_valeurs.head()

,insee,code_postal,ville,lat,lon,date,no2,o3,pm10,pm25,so2,indice
1,75101,75001,PARIS 01,48.861878,2.337414,2024-11-16,4,4,4,3,4,3
2,75101,75001,PARIS 01,48.861878,2.337414,2024-11-17,4,3,4,4,4,3
3,75102,75002,PARIS 02,48.867683,2.343129,2024-11-16,4,4,4,3,4,3
4,75102,75002,PARIS 02,48.867683,2.343129,2024-11-17,4,3,4,4,4,3
5,75103,75003,PARIS 03,48.862686,2.358687,2024-11-16,4,4,4,3,4,3


AFFICHAGE SUR LA CARTE JOUR PAR JOUR AVEC LA PARTICULE SOUHAITEE

In [137]:
fig = px.scatter_mapbox(data_valeurs,lat="lat", lon="lon",  hover_name="insee", height=700,color='pm25',size='pm25', animation_frame='date',
mapbox_style="open-street-map",title='On peut sortir ou pas ?',zoom=8,color_continuous_scale=["red","orange","yellowgreen","green"], opacity=0.2)
fig.update_layout(title_x=0.5)
fig.show()

ALLERGENES

In [138]:

url = "https://api.airparif.fr/pollens/bulletin"
headers = {'X-Api-Key': os.environ.get('airparif')}
p = {'insee' : '93010'}

response = requests.get(url, headers=headers)

In [139]:
response.json()

{'data': [{'taxons': ['cypres',
    'noisetier',
    'aulne',
    'peuplier',
    'saule',
    'frene',
    'charme',
    'bouleau',
    'platane',
    'chene',
    'olivier',
    'tilleul',
    'chataignier',
    'rumex',
    'graminees',
    'plantain',
    'urticacees',
    'armoises',
    'ambroisies'],
   'valeurs': {'75': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '77': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '78': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '91': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '92': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '93': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '94': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    '95': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]},
   'commentaire': "Clap de fin de saison pour les allergiques aux pollens qui vont pouvoir mieux respirer ces prochaines semain